In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from __future__ import print_function

from random import random
from datetime import datetime
import sys

import os
import pickle
import google.oauth2.credentials
from email.mime.text import MIMEText
import base64

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [ ]:
#set this to the name of the JSON file downloaded in step 7 above
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= ""

# Setup

In [ ]:
# If modifying these scopes, delete the file token_gmail.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.compose']

def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token_gmail.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token_gmail.pickle'):
        with open('token_gmail.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token_gmail.pickle', 'wb') as token:
            pickle.dump(creds, token)

if __name__ == '__main__':
    main()

In [ ]:
def get_authenticated_gmail_service():
    """
    This function uses the token_gmail.pickle file to get an authenticated object used to talk to Gmail
    """

    credentials = None
    API_SERVICE_NAME = 'gmail'
    API_VERSION = 'v1'
    
    if os.path.exists('token_gmail.pickle'):
        with open('token_gmail.pickle', 'rb') as token:
            credentials = pickle.load(token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [ ]:
service = get_authenticated_gmail_service()  

# Core Functions

In [ ]:
def create_message(sender, to, subject, message_text):
    """
    Create a message for an email.

    Args:
        sender: Email address of the sender.
        to: Email address of the receiver.
        subject: The subject of the email message.
        message_text: The text of the email message.

    Returns:
        An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode()).decode()}

In [ ]:
def send_message(service, user_id, message):
    """
    Send an email message.

    Args:
        service: Authorized Gmail API service instance.
        user_id: User's email address. The special value "me"
        can be used to indicate the authenticated user.
        message: Message to be sent.

    Returns:
        Sent Message.
    """
    
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print('Message Id: %s' % message['id'])
        return message
    except Exception as e:
        print('An error occurred: %s' % e)

# Run Code

In [ ]:
#create a message
my_message = create_message('your_email_address', 'your_email_address', 'Subject', 'message text')

In [ ]:
#send the message
send_message(service, 'me', my_message)